In [68]:
import pyrealsense2 as rs
import matplotlib as plt
import cv2
import logging
import numpy as np
import time

In [69]:
ctx = rs.context()
devices = ctx.query_devices()
len(devices)

2

In [70]:
for device in devices:
    print(device)

<pyrealsense2.device: Intel RealSense D435 (S/N: 923322071362)>
<pyrealsense2.device: Intel RealSense D435 (S/N: 923322070538)>


In [71]:
var = [i.get_info(rs.camera_info.serial_number) for i in devices]
var

['923322071362', '923322070538']

In [115]:
dev = devices[0].first_depth_sensor()
dev.set_option(rs.option.inter_cam_sync_mode,1)
dev.get_option(rs.option.inter_cam_sync_mode)

1.0

In [116]:
dev2 = devices[1].first_depth_sensor()
dev2.set_option(rs.option.inter_cam_sync_mode,2)
dev2.get_option(rs.option.inter_cam_sync_mode)

2.0

In [117]:
#cam1: configure depth and color streams
pipeline1 = rs.pipeline()
config1 = rs.config()
config1.enable_device(var[0])
config1.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 90)
config1.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 60)
config1.enable_record_to_file('master.bag')
print('cam1 enabled')

#cam2: configure depth and color streams 
pipeline2 = rs.pipeline()
config2 = rs.config()
config2.enable_device(var[1])
config2.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 90)
config2.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 60)
config2.enable_record_to_file('slave.bag')
print('cam2 enabled')

cam1 enabled
cam2 enabled


In [110]:
sync = rs.syncer(1)
sync
queue = rs.frame_queue(1, keep_frames = True)

In [118]:
pipeline1.start(config1)
pipeline2.start(config2)
print('cams configued')

cams configued


In [30]:
#timestamp in milliseconds
print(depthframe1.get_timestamp())
print(depthframe2.get_timestamp())

1571761838006.0913


RuntimeError: null pointer passed for argument "frame_ref"

In [31]:
#timestamp in uint32
depthframe1.get_frame_metadata(rs.frame_metadata_value.frame_timestamp)

21254441

In [67]:
for device in devices:
    rs.pipeline_profile.get_streams()
    streams = device.pipeline_profile.get_streams()
    frameset = device.pipeline.poll_for_frames()
    if frameset.size() == len(streams):
        frames[serial] = {}
        for stream in streams:
            if (rs.stream.depth == stream.stream_type()):
                frame = frameset.get_depth_frame(stream.stream_index())
                key_ = (stream.stream_type(), stream.stream_index())
            else:
                frame = frameset.first_or_default(stream.stream_type())
                key_ = stream.stream_type()
            frames[serial][key_] = frame
            

TypeError: get_stream(): incompatible function arguments. The following argument types are supported:
    1. (self: pyrealsense2.pyrealsense2.pipeline_profile, stream_type: pyrealsense2.pyrealsense2.stream, stream_index: int=-1) -> pyrealsense2.pyrealsense2.stream_profile

Invoked with: 

In [ ]:
def poll_frames():
    frames = {}
    while(len(frames) < len(devices)):
        for (serial, device) in devices:

In [119]:
try:
    start = time.time()
    while time.time() - start <10:
        
        #camera 1
        frames1 = pipeline1.wait_for_frames()
        depthframe1 = frames1.get_depth_frame()
        colframe1 = frames1.get_color_frame()
        if not depthframe1 or not colframe1:
            continue

        #convert images to numpy arrays
        depthim1 = np.asanyarray(depthframe1.get_data())
        colim1 = np.asanyarray(colframe1.get_data())
        
        #apply color map on depth image, convert to 8bit pixel
        depthcolmap1 = cv2.applyColorMap(cv2.convertScaleAbs(depthim1, alpha=0.5), cv2.COLORMAP_JET)
        
        #camera 2
        frames2 = pipeline2.wait_for_frames()
        depthframe2 = frames2.get_depth_frame()
        colframe2 = frames2.get_color_frame()
        if not depthframe2 or not colframe2:
            continue

            
        #check time sync
        #print([depthframe1, depthframe2]) #returns system_time
        counter = depthframe1.get_frame_metadata(rs.frame_metadata_value.frame_timestamp)
        counter2 = depthframe2.get_frame_metadata(rs.frame_metadata_value.frame_timestamp)
        timestamp1 = depthframe1.get_timestamp()
        timestamp2 = depthframe2.get_timestamp()
        print([counter,counter2, timestamp1, timestamp2, counter2-counter, timestamp2-timestamp1])
        #convert images to numpy arrays
        depthim2 = np.asanyarray(depthframe2.get_data())
        colim2 = np.asanyarray(colframe2.get_data())
        
        #apply color map on depth image, convert to 8bit pixel
        depthcolmap2 = cv2.applyColorMap(cv2.convertScaleAbs(depthim2, alpha=0.5), cv2.COLORMAP_JET)
        
        
        #stack all images horizontally
        images = np.hstack((colim1, depthcolmap1, colim2, depthcolmap2))
        
        #show images from cameras
        cv2.namedWindow('RealSense', cv2.WINDOW_NORMAL)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)
    
finally:
    #stop streaming
    pipeline1.stop()
    pipeline2.stop()
    cv2.destroyAllWindows()
    del pipeline1
    del config1
    del pipeline2
    del config2

[144989102, 147308055, 1571963895519.5247, 1571963895529.087, 2318953, 9.562255859375]
[145000235, 147397116, 1571963895530.6584, 1571963895618.155, 2396881, 87.49658203125]
[145167226, 147486178, 1571963895697.6633, 1571963895707.2239, 2318952, 9.560546875]
[145200624, 147497311, 1571963895731.064, 1571963895718.3577, 2296687, -12.706298828125]
[145245155, 147552974, 1571963895775.5986, 1571963895774.025, 2307819, -1.57373046875]
[145289686, 147608638, 1571963895820.1333, 1571963895829.6934, 2318952, 9.56005859375]
[145345349, 147642036, 1571963895875.801, 1571963895863.094, 2296687, -12.70703125]
[145378748, 147675434, 1571963895909.2026, 1571963895896.4944, 2296686, -12.708251953125]
[145401013, 147697700, 1571963895930.5457, 1571963895918.1755, 2296687, -12.3701171875]
[145423279, 147731098, 1571963895952.806, 1571963895951.5706, 2307819, -1.2353515625]
[145456677, 147764496, 1571963895986.195, 1571963895984.9656, 2307819, -1.2294921875]
[145478942, 147786762, 1571963896008.454, 15

[147382638, 149701590, 1571963897912.0005, 1571963897921.8413, 2318952, 9.8408203125]
[147404904, 149723856, 1571963897934.2625, 1571963897944.1047, 2318952, 9.84228515625]
[147438302, 149757254, 1571963897967.6545, 1571963897977.4988, 2318952, 9.84423828125]
[147460567, 149768387, 1571963897989.9155, 1571963897988.6306, 2307820, -1.284912109375]
[147482833, 149801785, 1571963898012.1775, 1571963898022.0247, 2318952, 9.84716796875]
[147516231, 149835183, 1571963898045.5693, 1571963898055.419, 2318952, 9.849609375]
[147549629, 149868581, 1571963898078.9614, 1571963898088.813, 2318952, 9.8515625]
[147560762, 149879714, 1571963898090.0923, 1571963898099.9446, 2318952, 9.852294921875]
[147583027, 149890847, 1571963898112.3533, 1571963898111.0764, 2307820, -1.27685546875]
[147605293, 149924245, 1571963898134.6155, 1571963898144.4705, 2318952, 9.85498046875]
[147638691, 149957643, 1571963898168.0073, 1571963898177.8645, 2318952, 9.857177734375]
[147683222, 149979909, 1571963898212.5303, 1571

[150199218, 152529303, 1571963900728.239, 1571963900748.9075, 2330085, 20.66845703125]
[150210350, 152540435, 1571963900739.3691, 1571963900760.0376, 2330085, 20.66845703125]
[150243748, 152562701, 1571963900772.7612, 1571963900782.2998, 2318953, 9.53857421875]
[150254881, 152584966, 1571963900783.8923, 1571963900804.5608, 2330085, 20.66845703125]
[150277147, 152596099, 1571963900806.1545, 1571963900815.692, 2318952, 9.537353515625]
[150310545, 152629497, 1571963900839.5466, 1571963900849.0842, 2318952, 9.53759765625]
[150321678, 152640630, 1571963900850.6777, 1571963900860.2153, 2318952, 9.53759765625]
[150332810, 152651763, 1571963900861.8079, 1571963900871.3464, 2318953, 9.53857421875]
[150366208, 152685161, 1571963900895.2002, 1571963900904.7388, 2318953, 9.53857421875]
[150377341, 152696293, 1571963900906.331, 1571963900915.839, 2318952, 9.508056640625]
[150399607, 152718559, 1571963900928.6326, 1571963900938.1008, 2318952, 9.46826171875]
[150421872, 152751957, 1571963900950.8938,

[152403497, 154722449, 1571963902932.2454, 1571963902941.6763, 2318952, 9.430908203125]
[152414630, 154733582, 1571963902943.3765, 1571963902952.807, 2318952, 9.430419921875]
[152448028, 154744715, 1571963902976.7683, 1571963902963.9377, 2296687, -12.83056640625]
[152470293, 154778113, 1571963902999.029, 1571963902997.329, 2307820, -1.699951171875]
[152481426, 154811511, 1571963903010.16, 1571963903030.7205, 2330085, 20.560546875]
[152503692, 154822644, 1571963903032.4219, 1571963903041.8513, 2318952, 9.429443359375]
[152537090, 154844909, 1571963903065.8137, 1571963903064.112, 2307819, -1.70166015625]
[152548223, 154867175, 1571963903076.9446, 1571963903086.3735, 2318952, 9.428955078125]
[152581621, 154900573, 1571963903110.3364, 1571963903119.765, 2318952, 9.428466796875]
[152603886, 154911706, 1571963903132.5974, 1571963903130.8958, 2307820, -1.70166015625]
[152637284, 154933971, 1571963903165.9893, 1571963903153.1565, 2296687, -12.832763671875]
[152648417, 154956237, 1571963903177.

[154552113, 156882198, 1571963905080.5762, 1571963905101.0747, 2330085, 20.49853515625]
[154585511, 156893331, 1571963905113.9678, 1571963905112.2053, 2307820, -1.762451171875]
[154596644, 156915596, 1571963905125.0986, 1571963905134.4656, 2318952, 9.366943359375]
[154618909, 156937862, 1571963905147.3594, 1571963905156.7268, 2318953, 9.367431640625]
[154630042, 156960127, 1571963905158.4902, 1571963905178.987, 2330085, 20.496826171875]
[154652308, 156971260, 1571963905180.7517, 1571963905190.1177, 2318952, 9.365966796875]
[154663440, 156982393, 1571963905191.8816, 1571963905201.2483, 2318953, 9.36669921875]
[154685706, 157004658, 1571963905214.1433, 1571963905223.5085, 2318952, 9.365234375]
[154719104, 157015791, 1571963905247.535, 1571963905234.6394, 2296687, -12.8955078125]
[154730237, 157038056, 1571963905258.6658, 1571963905256.8997, 2307819, -1.76611328125]
[154752502, 157071454, 1571963905280.9263, 1571963905290.2905, 2318952, 9.3642578125]
[154763635, 157082587, 1571963905292.0